# 06. PyTorch transfer learning

In [1]:
import torch
from torch import nn
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
print(torch.__version__)
print(torchvision.__version__)


KeyboardInterrupt



In [ ]:
try:
    from torchinfo import summary
except:
    print(f"[INFO] Couldn't found torchinfo... installing it")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to install Modular dir
try:
    from going_modular import data_setup, engine
except:
    print(f"[INFO] Couldn't found going_modular script... install it.")
    !git clone https://github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular .
    !rm -rf pytorch-deep-learning
    from going_modular import data_setup, engine

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
!nvidia-smi

## 1. Get Data

In [ ]:
import os
import zipfile
import requests
from pathlib import Path

data_path = Path("data_06/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
    print(f"{image_path} directory axists.")
else:
    print(f"Didn't found {image_path} downloding it")
    image_path.mkdir(parents=True, exist_ok=True)

    with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/data/pizza_steak_sushi.zip")
        print(f"Downloading....")
        f.write(request.content)

    # Unzip data
    with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
        print(f"Unziooing file....")
        zip_ref.extractall(image_path)
    # Remove Zip file
    os.remove(data_path / "pizza_steak_sushi.zip")

In [ ]:
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

## 2. Creating transform for `torchvision.models`
* All pre-trained models expect input images normalized in the same way, i.e. mini-batches of 3-channel RGB images of shape (3 x H x W), where H and W are expected to be at least 224.
The images have to be loaded in to a range of [0, 1] and then normalized using mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].

### 2.1 Manual creation

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
manual_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize]) # make sure images have the same distribution as ImageNet (where our pretrained mopdels have been trained)

In [ ]:
train_dataloader, test_dataloader, class_names = data_setup.create_dataloader(train_dir=train_dir, test_dir=test_dir, transforms=manual_transforms, batch_size=32)
train_dataloader, test_dataloader, class_names

### 2.2 Auto transform creation

In [ ]:
# Get a set of preptrained model weights
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # DEFAULT = best avalileble weights
weights

In [ ]:
# Get transfroms used to create our pretrained weights
auto_transfroms = weights.transforms()
auto_transfroms

In [ ]:
train_dataloder, test_datalodear, class_names = data_setup.create_dataloader(train_dir=train_dir, test_dir=test_dir, transforms=auto_transfroms, batch_size=32)
train_dataloader, test_dataloader, class_names

## 3. Getting pretrain modle

1. PyTorch main libraries
2. Libraries like `timm` (torch image models)
3. HoggingFase Hub
4. Paperswithcode 

### 3.1 Creating of a pretrained EffNet_B0

In [ ]:
weights

In [ ]:
model = torchvision.models.efficientnet_b0(weights=weights)
model

### 3.2 Get summary of our model with `torchinfo.summary()`

In [ ]:
summary(model=model, 
        input_size=(1, 3, 224, 224), 
        col_names=["input_size", "output_size", "num_params", "trainable"], 
        col_width=20, 
        row_settings=["var_names"])

### 3.3 Freezing the base model and changing the outup layer to suit our needs

In [ ]:
model.features

In [ ]:
# Freez all of the base layers in EffNeB0
for param in model.features.parameters():
    param.requires_grad = False

In [ ]:
# Updae the classifier head of our model to suit our problem
model.classifier

In [ ]:
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=1280, out_features=len(class_names)))

model.classifier

In [ ]:
summary(model=model, 
        input_size=(1, 3, 224, 224), 
        col_names=["input_size", "output_size", "num_params", "trainable"], 
        col_width=20, 
        row_settings=["var_names"])

## 4 Train model

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
from timeit import default_timer as timer

start_time = timer()
results = engine.train(model=model, 
                       train_dataloader=train_dataloader,
                       test_dataloader=test_datalodear,
                       loss_fn=loss_fn,
                       optimizer=optimizer,
                       epochs=5)
end_time = timer()
print(f"Total time: {end_time - start_time:.1f}. seconds")

## 5. Evaluate model by Plotting loss curves 

In [ ]:
try:
    from helper_function import plot_loss_curves
except:
    print(f"Couldn't find helper_function.py, downloading...")
    with open("helper_function.py", "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/refs/heads/main/helper_functions.py")
        f.write(request.contest)
    from helper_function import plot_loss_curves

plot_loss_curves(results=results)

## 6. Make predictions on images from the test set

In [ ]:
image_path

In [ ]:
from typing import Dict, List, Tuple
from PIL import Image

# 1. Take in a trained model...
def pred_and_plot_imge(model: torch.nn.Module,
                       img_path: str,
                       class_names: List[str],
                       image_size: Tuple[int, int] = (224, 224),
                       transform: torchvision.transforms = None,
                       device: torch.device = device):
    # 2. Open Image with PIL
    img = Image.open(img_path)

    # 3. Creat a transform
    if transform is not None:
        image_transform = transform
    else:
        image_transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
   
    model.to(device)

    # 4. Turn on inference_mode and eval mode
    model.eval()
    with torch.inference_mode():
        transformed_image = image_transform(img).unsqueeze(dim=0) # [batch_size, color_channales, H, W]
        traget_image_pred = model(transformed_image.to(device))
        
    target_image_pred_probs = torch.softmax(traget_image_pred, dim=1)
    target_imge_label = torch.argmax(target_image_pred_probs, dim=1)
    # 5 Plot image
    plt.figure()
    plt.imshow(img)
    plt.title(f"Pred: {class_names[target_imge_label]} | Prob: {target_image_pred_probs.max():.3f}")
    plt.axis(False)

In [ ]:
import random

num_images = 3

In [ ]:
test_images_path_list = list(Path(test_dir).glob("*/*.jpg"))
test_images_path_list

In [ ]:
test_image_path_random = random.sample(population=test_images_path_list, k=num_images)
test_image_path_random

In [ ]:
for img in test_image_path_random:
    pred_and_plot_imge(model=model,
                       img_path=img,
                       class_names=class_names)

### 6.1 Custom image Prediction 

In [ ]:
custom_image_path = data_path / "04-pizza-dad.jpeg"

if not custom_image_path.is_file():
    with open(custom_image_path, "wb") as f:
        request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/images/04-pizza-dad.jpeg")
        f.write(request.content)
else:
    print(f"{custom_image_path} alredy exists")

In [ ]:
pred_and_plot_imge(model=model,
                   img_path=custom_image_pathe,
                   class_names=class_names)